# Analysis
In this notebook we will talk about our findings from this project, include various animations of both 2d birds,3d birds and predator prey models flocking and plots of how the Vicsek Order Parameter and convergence of this overdamped system is affected by varying the parameters. To accompany this, we made an interactive animation where the birds flock to or away from the cursor, and a 3d plot which allows you to move around the spatial dimension.

**Coding Style**

To code the Vicsek Model we opted to use a Object Orientated Programming framework. By creating different objects, we were able to group useful functions together which improves readability and efficiency in the code. We use matplotlib FuncAnimations of quiver plots to visualise the movement of the birds.  

**Requirements**

Numpy and Pandas were used for data handling.  
Matplotlib was predominantly used for visualisation, alongside Plotly and Pygame for some interactive plots.  
HTML from IPythonDisplay is used to embed animations in the notebook. Alternatively rc("animation","html5") could be used.  


In [ ]:
from flock import *

## 2D Birds
**The Vicsek Model**

The Visek Model is a simple model of flocking behaviour proposed by Tamas Vicsek, which uses the following equations in discrete time to update the position of each bird.
$$
\text{$\vec{r}(k+1)_i$ = $\vec{r}(k)_i$ + $\hat{n_i}v_0$}
$$
$$
\text{$\theta_i$(k+1) = angle [$\sum_{j=1}^{z_i}$}\hat{n_j}] + \eta_i
$$

**The Flock Class**

This is our main class, from which many different types of bird such as moth,prey and predator inherit. When initialised it creates uniformly random positions in a (frame_size,frame_size) frame where frame_size is given in number of radiuses, and random directions with angle uniformly distributed in $[0,2\pi)$. We interact with it mainly through the display methods such as display_state, animate_movement, and plot_order_stat. The main internal method is update_posdirs which syncronously updates positions and directions according to the Vicsek Model. 

**Implementing Periodic Boundary Conditions**

To wrap the frame we used a method suggested by Dr Henkes. By tiling the birds positions together we create two 3d arrays, one tiling the list of positions horizontally and the other vertically. By subtracting the two 3d arrays we create an array of the pairwise vectors between each bird (Flock.pvec()). This function is not disimilar to pdist in scipy.special.distance() with the difference being that the array we create is of vectors not distances. From this we can then calculate if each component of each vector is bigger than (L/2), where L is the frame size. If this is true we then recalculate the component now being itself modulo the frame size, effectively wrapping the vectors the other way round the torus the birds lie on. 

**Vectorisation**

Through vectorising all calculations using numpy we were able to improve run times from our first for loop based methods by several orders of magnitude. This even allowed us to later create an interactive game with Moths and Prey following the cursor in real time.

**Animation**

We were able to get some good looking animations of flocking using matplotlib.quiver plots alongside FuncAnimations. We will render one below with the recommended paramaters of 200 birds, speed 0.5, in a container 15 times their radius of vision.

In [ ]:
##Generate a flock of 200 birds, speed 0.5, frame size 15 radii
f = Flock(200,0.5,15)
matplotlib.rcParams['animation.embed_limit'] = 100
anim_03_1000 = f.animate_movement(1000,0.3)
HTML(anim_03_1000.to_jshtml())

In [ ]:
##now lets add some more birds
##Generate a flock of 200 birds, speed 0.5, frame size 15 radii
f = Flock(500,0.5,20)
matplotlib.rcParams['animation.embed_limit'] = 100
anim_03_1000 = f.animate_movement(1000,0.3)
HTML(anim_03_1000.to_jshtml())

## Vicsek Order Paramater 
Below we plot the Vicsek Order Parameter and consider the implications for the model in terms of phase transitions.  

**Vicsek Order Paramater**

We calculate the Vicsek Order Paramater using 
$$
\text{n = $\frac{1}{N}$|$\sum_{i=1}^{N}$}\hat{n_i}|.
$$
This is essentially the normalised sum of directions of all the birds, varying between 0 and 1 where 0 represents complete polar disorder and 1 represents complete polar symmetry, ie all birds are pointing in the same direction. We will plot how this changes alongside some animations of the birds moving, as well as some plots with error bars for multiple runs. 

In [ ]:
##reseting the positions to random positions and generating side by side animations of the order stat and birds flocking for sigma =0.1 and sigma = 1
f = Flock(200,0.5,15)
anim_vop_01_200 = f.animate_movement(200,0.1,plot_order_stat= True)
HTML(anim_vop_01_200.to_jshtml())

In [ ]:
f.reset()
anim_vop_1_200 = f.animate_movement(200,1,plot_order_stat= True)
HTML(anim_vop_1_200.to_jshtml())

In [ ]:
file_name = r"C://Users//bennm//Documents//UNI//Year 2//Mathematical Programming//Project//vicsek-model-group3//vicsek_flocking_vop_1_200.mp4"
writervideo = animation.FFMpegWriter(fps=30) 
anim_vop_1_200.save(file_name, writer=writervideo)

In [ ]:
def plot_order_stat_EB(sigma,B=10,T=200):
    """Plots the order stat against time for B runs of period T, with an error bar plot on the right"""
    ##generating the data
    f = Flock(200,0.5,15)
    ts = np.zeros((B,T))
    vops =np.zeros((B,T))
    for i in range(B):
        ts[i],vops[i] = f.plot_order_stat(1,T,sigma,display=False,plot=False)
        f.reset()

    ##setting up plot1
    fig,(ax1,ax2) = plt.subplots(1,2)
    fig.set_size_inches(14,7)
    ax1.plot(ts[0],vops.transpose())
    ax1.set_ylabel('Order Statistic')
    ax1.set_xlabel('Time Steps')
    ax1.set_title(f'Order statistic plot (10 samples) for sigma = {sigma}',fontsize=15)
    ax2.plot(ts[0],np.mean(vops.transpose(),axis=1))

    ##setting up plot2
    yerr = np.std(vops.transpose(),axis=1)
    ax2.errorbar(ts[0],np.mean(vops.transpose(),axis=1),
            xerr=0,
            yerr=yerr,
            errorevery=20,
            ecolor = 'b')
    ax2.set_ylabel('Order Statistic',fontsize = 10)
    ax2.set_xlabel('Time Steps')
    ax2.set_title(f'Order statistic plot with error bars for sigma = {sigma}',fontsize=15)
 
 

In [ ]:
##plotting error bar plots for 5 different sigma
plot_order_stat_EB(0)
plot_order_stat_EB(0.5)
plot_order_stat_EB(1)
plot_order_stat_EB(1.5)
plot_order_stat_EB(2)

## $n(\sigma)$ Phase Transition Plot
The code below was used to generate 10 samples of 1000 steps for each noise, the first 200 steps were removed and the average order statistic was calulated. We then filled out a Pandas dataframe and saved as a csv for plotting. This takes a significant amount of time to run (about 40 minutes).

In [ ]:

# nrun = 20 
# nsig = 20.  #Running for 20 different sigma, 20 runs of each
# data = np.zeros((nrun,nsig)). #empty array to fill out
# for i in range(nrun):
#     print(f'completed run {i}') #progress tracker
#     for j in range(nsig): #double for loop, looping 20 runs for each sigma
#         f.reset() # resets the flock after each run
#         ts,vops=f.plot_order_stat(1,1000,j*0.1,display=False,plot=False) #vops is a list of our order statistic at each step
#         data[i,j] = np.mean(vops[200:]) #calculates the mean of our order statistic without the first 200 steps
# print(data)        


In [ ]:
# colname = np.array(np.round(np.linspace(0,1.9,20), decimals = 1),dtype= str)

# ##create a data frame from the above loop and save as a csv
# df = pd.DataFrame(data, columns = colname)
# f = open('data.csv',"w+")
# f.write(df.to_csv(index = True)) 
# f.close()




In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/bennm37/vicsek-model-group3/animate_vicsek_birds/data.csv') #pulls our csv from github
df
datmean = np.array(df.mean())
datst = np.array(df.std())    #calculates the mean and standard deviation of our data for each sigma  
x = np.linspace(0,1.9,20) # x data is our sigma which can be plotted as linspace
y = datmean # y data is the mean order statstic for each sigma
yerr = datst # our y error is the standard deviation for each sigma

fig, ax = plt.subplots()

ax.errorbar(x, y,
            xerr=0,
            yerr=yerr,
            )              #plotted using matplot lib with error bars              
ax.set_xlabel('$\sigma$')
ax.set_ylabel('Order Statistic')
ax.set_title('$n(\sigma)$ Phase Transition Plot')


plt.show()




From plotting the $n(\theta)$ phase transition diagram we can infer characteristics about flocking in the Vicsek model. We can see that when noise is low, we observe the most flocking as the order statistic is highest, conversely when we have very little flocking when noise exceeds 1.25. The plot plateaus in both ends with a linear looking mid-section. In the tails of our plot we have very small error bars meaning the average order statistic of the flock is more consistent hence when our noise takes an extreme value the flock will almost surely end up either flocked or not at all.

## 3D Birds 
By adapting the Flock class we have created Flock_3d that updates, displays and animates 3d birds. To initate their directions we considered projecting uniformly random points on a cylinder onto a sphere to get an even distribution of points unbiased about the poles. Positions were uniformly random in $\mathbb{R}^3$. Here 6 radii are used instead of 15, as a paper(cite) suggested an important parameter was the ratio of the number of birds to the volume of the frame in radii. 



**3d Noise Generation**

 To update directions we used noise that was distributed uniformly in theta and normally in phi. Although this is not strictly normal about the pole as points uniform on the polar angle are not uniform on the surface of the sphere, it is azimodially symmetrical and will suffice for the purposes of this project. We plot noise for different sigmas below.

In [ ]:
##creating a 3d flock and plotting noise for different sigma
f_3d = Flock_3d(1000,0.5,6)
n1 = f_3d.get_noise(0.1)
n2 = f_3d.get_noise(0.5)
##significantl higher noise levels lose their meaning as the whole sphere is covered
n3 = f_3d.get_noise(1.25)

fig = plt.figure()
fig.set_size_inches(15,10)
# ax1.set_title("Noise Plots for sigma = 0.1,0.5 and 1.5",fontsize=10)
ax1= fig.add_subplot(1,3,1,projection="3d")
ax2= fig.add_subplot(1,3,2,projection="3d")
ax3= fig.add_subplot(1,3,3,projection="3d")
ax1.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax2.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax3.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax1.scatter(n1[:,0],n1[:,1],n1[:,2])
ax2.scatter(n2[:,0],n2[:,1],n2[:,2])
ax3.scatter(n3[:,0],n3[:,1],n3[:,2])


We can quickly check all these directions lie on a unit sphere by calulating the norm of each.

In [ ]:
assert np.allclose(lag.norm(n1,axis=1),1)
assert np.allclose(lag.norm(n2,axis=1),1)

**Adding the Noise**

Having generated the noise, we needed to rotate and scale it to each sum of directions in the current birds radius before adding it on. We did this by getting the spherical coordinates of each direction sum, and rotating the noise using get_rotation_matrix method of Flock_3d, which calculates the rotation in the $xy$ plane by $\theta$ and the rotation in $xz$ plane by $\phi$ using the rotation matrices below.  
$$
\text{Rotation matrix used to rotate vectors by angle $\theta$ in the XY plane}
$$
$$
\begin{bmatrix} 
cos(\theta) & sin(\theta) & 0 \\
-sin(\theta) & cos(\theta) & 0\\
0 & 0 & 1 \\
\end{bmatrix}
\quad
$$
$$
\text{Rotation matrix used to rotate vectors by angle $\phi$ in the XZ plane}
$$
$$
\begin{bmatrix} 
cos(\phi) & 0 & -sin(\phi) \\
0 & 1 & 0\\
sin(\phi) & 0 & cos(\phi) \\
\end{bmatrix}
\quad
$$  
We'll plot 3 rotations we used to check whether the noise rotation was working effectively.

In [ ]:
ds = np.array([[0,0,1],[1,1,1],[-1,0,0]])
rs = f_3d.get_rotation_matrices(ds)
nd = np.zeros((3,n1.shape[0],3))
for j in range(3):
    for i in range(n1.shape[0]):
        nd[j,i,:] = np.array([np.matmul(rs[j],n1[i])])

##setting up the plots
fig = plt.figure()
fig.set_size_inches(15,10)
# ax1.set_title("Noise Plots for sigma = 0.1,0.5 and 1.5",fontsize=10)
ax1= fig.add_subplot(1,3,1,projection="3d")
ax2= fig.add_subplot(1,3,2,projection="3d")
ax3= fig.add_subplot(1,3,3,projection="3d")
ax1.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax2.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax3.set(xlim=(-1,1),ylim=(-1,1),zlim=(-1,1))
ax1.scatter(nd[0,:,0],nd[0,:,1],nd[0,:,2])
ax2.scatter(nd[1,:,0],nd[1,:,1],nd[1,:,2])
ax3.scatter(nd[2,:,0],nd[2,:,1],nd[2,:,2])

**3d animation**

The majority of other methods scaled nicely to do 3d with some minor changes. Sadly 3d quiver plots in matplotlib don't support animation in the way we would like, so we had to resort to scatter plots to visualise the birds moving. Some renders for different sigmas are presented below.

In [ ]:
##creating a 3d Flock with 400 birds, speed 0.5 in a 8x8x8 box.
f_3d = Flock_3d(400,0.5,8)
anim_3d_02_200 = f_3d.animate_movement(0.2,200)
matplotlib.rcParams['animation.embed_limit'] = 100
HTML(anim_3d_02_200.to_jshtml())

In [ ]:
##now lets add an ambient camera rotation and try a different sigma 
f_3d = Flock_3d(400,0.5,6)
anim_3d_08_200 = f_3d.animate_movement(0.8,200,ambient_rotation =True)
HTML(anim_3d_08_200.to_jshtml())

Sadly these animations are not particularly aesthetic or reprensentative of the flocking. To get some better looking plots that clearly demonstate flocking we turned to the Plotly library.

In [ ]:
f_3d = Flock_3d(500,0.5,15)
for i in range(100):
    f_3d.update_posdirs(1,0.1)
f_3d.display_state_plotly()

## Moths and Prey  
Inheriting from the flock class we created 'Prey' and 'Moth'. These update like normal vicsek birds but with an additional attraction or repulsion factor, calculated by $\frac{1}{|v|}v$ where $v$ is a vector between predator and prey. This vectorises nicely in a similar way to Flock.pvec. Through vectorising all operations we were able to make this update in real time for an interactive animation in which the birds follow or run from the cursor. To see the Moth functionality press m, and Prey functionality press p. To change the background colour press b.


In [ ]:
%run -m birds_pygame

We also created some quiver plots of multiple prey and predators chasing each other. The predator class has very similar functionality to Moth. 

In [ ]:
prey = Prey(200,0.5,15)
pred = Predator(10,0.5,15)
anim_pp_05_200 = pred.animate_movement_pp(prey,200,0.5,60)
HTML(anim_pp_05_200.to_jshtml())